In [60]:
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np

mnist = tf.keras.datasets.mnist
layer_amt = 3

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

base_model = tf.keras.models.Sequential()
base_model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

for x in range(layer_amt): 
    base_model.add(tf.keras.layers.Dense(128, activation='relu'))

base_model.add(tf.keras.layers.Dropout(0.2))
base_model.add(tf.keras.layers.Dense(10))


base_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
base_model.fit(x_train, y_train, epochs=15)

Epoch 1/15
1875/1875 [==============================] - 2s 852us/step - loss: 0.2422 - accuracy: 0.9273
Epoch 2/15
1875/1875 [==============================] - 1s 795us/step - loss: 0.1037 - accuracy: 0.9698
Epoch 3/15
1875/1875 [==============================] - 1s 787us/step - loss: 0.0773 - accuracy: 0.9769
Epoch 4/15
1875/1875 [==============================] - 2s 804us/step - loss: 0.0584 - accuracy: 0.9821
Epoch 5/15
1875/1875 [==============================] - 2s 819us/step - loss: 0.0485 - accuracy: 0.9848
Epoch 6/15
1875/1875 [==============================] - 2s 821us/step - loss: 0.0417 - accuracy: 0.9869
Epoch 7/15
1875/1875 [==============================] - 2s 800us/step - loss: 0.0359 - accuracy: 0.9888
Epoch 8/15
1875/1875 [==============================] - 2s 818us/step - loss: 0.0313 - accuracy: 0.9900
Epoch 9/15
1875/1875 [==============================] - 2s 844us/step - loss: 0.0261 - accuracy: 0.9918
Epoch 10/15
1875/1875 [==============================] - 2s 821u

In [61]:
base_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.1117 - accuracy: 0.9778 - 140ms/epoch - 448us/step


[0.11166907101869583, 0.9778000116348267]

In [62]:
base_model.save('base_model.keras')

In [63]:
numpy_array = np.random.rand(300,1000)
np.save("array.npy", numpy_array)

In [64]:
# what we need
merge_files('base_model.keras', "array.npy", destination="blahblah.optimized")

# custom file type "blahblah.optimized"
with open("blahblah.optimized") as f: 
    base_model, numpy_array = unmerge_files("blahblah.optimized")

NameError: name 'merge_files' is not defined

In [65]:
import struct
import io

def merge_binary_files(file_a, file_b, file_c):
    with open(file_a, 'rb') as fa, open(file_b, 'rb') as fb:
        content_a = fa.read()
        content_b = fb.read()
        
    # two integer representation 
    metadata = struct.pack('QQ', len(content_a), len(content_b))

    # Write metadata 
    with open(file_c, 'wb') as fc:
        fc.write(metadata)
        fc.write(content_a)
        fc.write(content_b)
    return file_c

def unmerge_binary_file(merged_file, output_file_a, output_file_b):
    with open(merged_file, 'rb') as f:
        metadata = f.read(struct.calcsize('QQ'))
        size_a, size_b = struct.unpack('QQ', metadata)
        content = f.read()
        content_a = content[:size_a]
        content_b = content[size_a:]
    with open(output_file_a, 'wb') as fa, open(output_file_b, 'wb') as fb:
        fa.write(content_a)
        fb.write(content_b)
    return tf.keras.models.load_model(output_file_a), np.load(output_file_b)
# Example usage
merge_binary_files('base_model.keras', 'array.npy', 'merged_file.optimized')
comp_model, comp_array = unmerge_binary_file('merged_file.optimized', 'base_model_res.keras', 'array_res.npy')

In [71]:
comp_array.shape


(300, 1000)

In [67]:
comp_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.1117 - accuracy: 0.9778 - 186ms/epoch - 593us/step


[0.11166907101869583, 0.9778000116348267]

In [68]:
import filecmp
filecmp.cmp("base_model_res.keras", "base_model.keras")

True

In [69]:
filecmp.cmp("array.npy", "array_res.npy")

True

In [70]:
filecmp.cmp("base_model_res.keras", "array_res.npy")

False